In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib as plt
%matplotlib inline
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split

#Split values based on the two CSVs
training = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv")
testing = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv")

data = training
target = data['Survived'] 

In [5]:
# Show percentage survivors by sex and by class
pvt_train = pd.pivot_table(data=training, index=['Survived'], columns=['Sex', 'Embarked'], values =['PassengerId'], aggfunc='count')
pvt_train.apply(lambda x: x / (x.max() + x.min()))

PassengerId                                              
Sex           female                      male                    
Embarked           C     Q         S         C         Q         S
Survived                                                          
0           0.123288  0.25  0.310345  0.694737  0.926829  0.825397
1           0.876712  0.75  0.689655  0.305263  0.073171  0.174603

In [91]:
# We will try to loop through the different classes and sex to apply the median age
for sex in xrange(0,2):
    for cabin_class in xrange(0,4):
        median_age_group = features[(features.Pclass == cabin_class) & (features.Sex_female == sex)].dropna().median()
        features[(features.Pclass == cabin_class) 
                 & (features.Sex_female == sex)] = features[(features.Pclass == cabin_class) 
                                                                     & (features.Sex_female == sex)
                                                                    ].fillna(median_age_group)
# adding validation for if the person is a child
features['child'] = (features.Age <= 18)*1

In [93]:
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import BernoulliNB

mnb = BernoulliNB()

params = {
    'alpha': [0.005, 0.0051, 0.00059],
}
mnbgs = GridSearchCV(mnb, params, cv=5, scoring='roc_auc')
mnbgs.fit(features, target)

print mnbgs.best_score_
print mnbgs.best_params_

0.799451201536
{'alpha': 0.005}


In [94]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

rf = RandomForestClassifier(n_estimators=100)

params = {
    'max_features': [0.75, 0.8, 0.9],
    'max_depth': [3.75, 3.9, 4, 4.1],
}
rgs = GridSearchCV(rf, params, cv=5, scoring='roc_auc')
rgs.fit(features, target)
print rgs.best_score_
print rgs.best_params_

0.868623852281
{'max_features': 0.9, 'max_depth': 4}


In [79]:
print rgs.best_score_
print rgs.best_params_

0.866916607304
{'max_features': 0.75, 'max_depth': 4.1}


In [96]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

gb = GradientBoostingClassifier(n_estimators=100, subsample=.8)

params = {
    'learning_rate': [0.05, 0.1, 0.2, 0.5],
    'max_features': [0.7, 0.75, 0.8],
    'max_depth': [ 3.5, 3.6, 3.75, 3.9, 4],
}
gs = GridSearchCV(gb, params, cv=10, scoring='roc_auc')
gs.fit(features, target)
print gs.best_score_
print gs.best_params_

0.877983240881
{'max_features': 0.8, 'learning_rate': 0.2, 'max_depth': 3.75}


In [61]:
print gs.best_score_
print gs.best_params_

0.878693523426
{'max_features': 0.75, 'learning_rate': 0.1, 'max_depth': 3.5}


In [203]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

# [['Pclass','SibSp','Parch','child','Survived','sex_bool']]
data = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv")
data['embarked_val'] = data['Embarked'].dropna().map({'S': 1, 'C': 2, 'Q': 3}).astype(int)
data['child'] = (data.Age.dropna() <= 18)*1
data['sex_bool'] = (data.Sex.dropna() == 'male')*1
cleaned_data = data[['Pclass','child','Survived','sex_bool','embarked_val']].dropna() 
X = cleaned_data[['Pclass','child','sex_bool','embarked_val']]
y = cleaned_data[['Survived']]

X_train, X_test, y_train, y_test = train_test_split(X, y)

#-------------------------------------------------------------------
# pipelines and scalars
#-------------------------------------------------------------------

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA, RandomizedPCA


In [26]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [16]:
#data['expensive_coach_woman'] = ((data.Sex == 'female') & (data.Pclass==3) & (data.Fare >= 20)) * 1
pvt_train = pd.pivot_table(data=data, index=['Survived'], columns=['Family_size'], values =['PassengerId'], aggfunc='count')
pvt_train.apply(lambda x: x / (x.max() + x.min()))

PassengerId                                               \
Family_size          0         1         2         3    4         5    
Survived                                                               
0              0.696462  0.447205  0.421569  0.275862  0.8  0.863636   
1              0.303538  0.552795  0.578431  0.724138  0.2  0.136364   

                                 
Family_size        6    7    10  
Survived                         
0            0.666667  0.5  0.5  
1            0.333333  NaN  NaN

In [5]:
#data['child'] = (data.Age.dropna() <= 18)*1
#feature engineering
data['expensive_coach_woman'] = ((data.Sex == 'female') & (data.Pclass==3) & (data.Fare >= 20)) * 1
# based on looking at the values there are a few that appear very infrequently, we will map them to more useful values
title_map = {'Capt': 'Sir', 'Don':'Sir', 'Major':'Sir', 'Sir':'Sir', 'Col':'Sir', 'Mlle':'Sir', 'Jonkheer':'Sir',
             'Mme': 'Lady', 'Lady':'Lady', 'the Countess':'Lady', 'Ms': 'Miss'}
def title_extract(name):
    title = name.split(',')[1].split('.')[0].strip()
    if title in title_map:
        title = title_map[title]
    return title
data['Title'] = data.Name.apply(lambda x: title_extract(x))

data['Surname'] = data.Name.apply(lambda x: x.split(',')[0])
data['Family_size'] = data.SibSp + data.Parch + 1
data['FamilyID'] = (data.Family_size.astype(str) + data.Surname) * ((data.Family_size >= 2) * 1)
data['FamilyID'] = data['FamilyID'].apply(lambda x: 'small' if x == '' else x)

features = pd.concat([data[['Fare', 'Age', 'expensive_coach_woman', 'Family_size']],
                      pd.get_dummies(data['Sex'], prefix='Sex'),
                      pd.get_dummies(data['Pclass'], prefix='Pclass'),
                      pd.get_dummies(data['Title']),
                      pd.get_dummies(data['Embarked'], prefix='Embarked'), 
                     ],
                     axis=1)
features = features.drop('Sex_male', 1)

# Because of the following bug we cannot use NaN as the missing
# value marker, use a negative value as marker instead:
# https://github.com/scikit-learn/scikit-learn/issues/3044
features = features.fillna(-1)
features.head(5)

,Fare,Age,expensive_coach_woman,Family_size,Sex_female,Pclass_1,Pclass_2,Pclass_3,Dr,Lady,Master,Miss,Mr,Mrs,Rev,Sir,Embarked_C,Embarked_Q,Embarked_S
0,7.2500,22,0,2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
1,71.2833,38,0,2,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0
2,7.9250,26,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1
3,53.1000,35,0,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1
4,8.0500,35,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1


In [26]:
pca = RandomizedPCA()
pca.fit(pd.concat([pd.DataFrame(target), features], axis=1))

RandomizedPCA(copy=True, iterated_power=3, n_components=None,
       random_state=None, whiten=False)

In [28]:
print pca.explained_variance_ratio_

[  8.86529590e-01   1.11927831e-01   8.81734402e-04   2.20437110e-04
   1.14540870e-04   9.51323119e-05   5.90607733e-05   5.38733937e-05
   4.03863872e-05   3.36858247e-05   2.18496721e-05   8.90949415e-06
   4.94619979e-06   3.25734262e-06   2.50446361e-06   1.65305541e-06
   3.45522667e-07   2.62309355e-07   1.19070628e-36   2.25383453e-37]


In [38]:
# Minimum percentage of variance we want to be described by the resulting transformed components
variance_pct = .99
 
# Create PCA object
pca = PCA(n_components=variance_pct)
 
# Transform the initial features
X_transformed = pca.fit_transform(features.values, target)
 
# Create a data frame from the PCA'd data
pcaDataFrame = pd.DataFrame(X_transformed)
 
print pcaDataFrame.shape[1], " components describe ", str(variance_pct)[1:], "% of the variance"

2  components describe  .99 % of the variance


In [31]:
imputer = Imputer(strategy='median', missing_values=-1)
scalar = StandardScaler()
pca = PCA()
classifiers = [RandomForestClassifier(), GradientBoostingClassifier(), 
               ExtraTreesClassifier(), LogisticRegression(), DecisionTreeClassifier(),
               KNeighborsClassifier(), SVC(), LinearSVC(), GaussianNB()]

print '\n'
for classifier in classifiers:
    pipeline = Pipeline([
        #('scl', scalar),
        ('imp', imputer),
        ('clf', classifier),
    ])
    #scores = cross_val_score(pipeline, features.values, target, cv=5, scoring='accuracy')
    scores = cross_val_score(pipeline, pcaDataFrame, target, cv=5, scoring='accuracy')
    print str(classifier)
    print('min -', scores.min(), 'mean -', scores.mean(), 'max -', scores.max())
    print '\n\n'



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
('min -', 0.5977653631284916, 'mean -', 0.67129747743357926, 'max -', 0.7078651685393258)



GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
('min -', 0.62011173184357538, 'mean -', 0.69923001276337537, 'max -', 0.7415730337078652)



ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max

## Gradient Boosting

In [29]:
imputer = Imputer(strategy='median', missing_values=-1)
scalar = StandardScaler()
pca = PCA()

classifier = GradientBoostingClassifier(n_estimators=100, subsample=.8)

params = {
    'pca__n_components': np.arange(2,features.shape[1]+1).tolist() ,
    'clf__learning_rate': [0.1, 0.15, 0.2],
    'clf__max_features': [0.76, 0.77, 0.78],
    'clf__max_depth': [3.6, 3.63],
}

pipeline = Pipeline([
    #('scl', scalar),
    ('imp', imputer),
    ('pca', pca),
    ('clf', classifier),
])

grid_search = GridSearchCV(pipeline, params, cv=10, scoring='roc_auc')
grid_search.fit(features.values, target)
print grid_search.best_score_
print grid_search.best_params_


0.873464313
{'clf__max_features': 0.78, 'clf__max_depth': 3.6, 'clf__learning_rate': 0.1, 'pca__n_components': 8}


## Random Forest

In [ ]:
imputer = Imputer(strategy='median', missing_values=-1)
scalar = StandardScaler()
pca = PCA()

classifier = RandomForestClassifier(n_estimators=100)

params = {
    'pca__n_components': np.arange(2,features.shape[1]+1).tolist() ,
    'clf__criterion':['gini','entropy'],
    'clf__max_features': [0.75, 0.8, 0.9],
    'clf__max_depth': [3.75, 3.9, 4, 4.1],
    'clf__min_samples_split': [2, 10, 50],
}

pipeline = Pipeline([
    ('scl', scalar),
    ('imp', imputer),
    ('pca', pca),
    ('clf', classifier),
])

grid_search2 = GridSearchCV(pipeline, params, cv=10, scoring='roc_auc')
grid_search2.fit(features.values, target)
print grid_search2.best_score_
print grid_search2.best_params_


In [4]:
imputer = Imputer(strategy='median', missing_values=-1)
scalar = StandardScaler()
pca = PCA()

classifier = RandomForestClassifier(n_estimators=100)
min_samples_split=15, n_estimators=73, criterion='entropy'
params = {
    'pca__n_components': np.arange(2,features.shape[1]+1).tolist() ,
    'clf__criterion':['gini','entropy'],
    'clf__max_features': [0.75, 0.8, 0.9],
    'clf__max_depth': [3.75, 3.9, 4, 4.1],
    'clf__min_samples_split': [2, 10, 50],
}

pipeline = Pipeline([
   #('scl', scalar),
    ('imp', imputer),
    ('pca', pca),
    ('clf', classifier),
])

grid_search = GridSearchCV(pipeline, params, cv=10, scoring='roc_auc')
grid_search.fit(features.values, target)
print grid_search.best_score_
print grid_search.best_params_

ValueError: could not convert string to float: small

In [28]:
imputer = Imputer(strategy='median', missing_values=-1)
scalar = StandardScaler()
pca = PCA()

classifier = LinearSVC()

params = {
    'pca__n_components': np.arange(2,features.shape[1]+1).tolist() ,
    #'clf__penalty':['l2','l1'],
    #'clf__loss': ['hinge','squared_hinge'],
    'clf__class_weight': [None, 'auto'],
    'clf__C': [1.0, 10.0, 100.0, 1000.0],
}

pipeline = Pipeline([
    ('scl', scalar),
    ('imp', imputer),
    ('pca', pca),
    ('clf', classifier),
])

grid_search = GridSearchCV(pipeline, params, cv=10, scoring='roc_auc')
grid_search.fit(features.values, target)
print grid_search.best_score_
print grid_search.best_params_

0.867379761709
{'pca__n_components': 15, 'clf__C': 1.0, 'clf__class_weight': None}


# Predicting Test Values for Kaggle Submission

In [31]:
testing['expensive_coach_woman'] = ((testing.Sex == 'female') & (testing.Pclass==3) & (testing.Fare >= 20)) * 1
# based on looking at the values there are a few that appear very infrequently, we will map them to more useful values
title_map = {'Capt': 'Sir', 'Don':'Sir', 'Major':'Sir', 'Sir':'Sir', 'Col':'Sir', 'Mlle':'Sir', 'Jonkheer':'Sir',
             'Mme': 'Lady', 'Lady':'Lady', 'the Countess':'Lady', 'Ms': 'Miss'}
def title_extract(name):
    title = name.split(',')[1].split('.')[0].strip()
    if title in title_map:
        title = title_map[title]
    return title
testing['Title'] = testing.Name.apply(lambda x: title_extract(x))

testing['Surname'] = testing.Name.apply(lambda x: x.split(',')[0])
testing['Family_size'] = testing.SibSp + testing.Parch + 1
testing['FamilyID'] = (testing.Family_size.astype(str) + testing.Surname) * ((testing.Family_size >= 2) * 1)
testing['FamilyID'] = testing['FamilyID'].apply(lambda x: 'small' if x == '' else x)

testing_features = pd.concat([testing[['Fare', 'Age', 'expensive_coach_woman', 'Family_size']],
                      pd.get_dummies(testing['Sex'], prefix='Sex'),
                      pd.get_dummies(testing['Pclass'], prefix='Pclass'),
                      pd.get_dummies(testing['Title']),
                      pd.get_dummies(testing['Embarked'], prefix='Embarked')
                     ],
                     axis=1)
testing_features = testing_features.drop('Sex_male', 1)

# Because of the following bug we cannot use NaN as the missing
# value marker, use a negative value as marker instead:
# https://github.com/scikit-learn/scikit-learn/issues/3044
testing_features = testing_features.fillna(-1)
testing_features.head(5)

,Fare,Age,expensive_coach_woman,Family_size,Sex_female,Pclass_1,Pclass_2,Pclass_3,Dona,Dr,Master,Miss,Mr,Mrs,Rev,Sir,Embarked_C,Embarked_Q,Embarked_S
0,7.8292,34.5,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,7.0000,47.0,0,2,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1
2,9.6875,62.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
3,8.6625,27.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
4,12.2875,22.0,0,3,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1


In [32]:
pred = grid_search.predict(testing_features)

gb_pred = pd.concat([testing['PassengerId'], pd.DataFrame(data=pred, columns=['Survived'])], axis=1)
gb_pred.to_csv('GB_classifier_20150705.csv', index=False)

In [1]:
import numpy as np

import pandas as pd

from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, Imputer
from patsy import dmatrices, dmatrix

#Print you can execute arbitrary python code
#Split values based on the two CSVs
df_train = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv", 
                       dtype={"Age": np.float64} )
df_test = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv", 
                    dtype={"Age": np.float64} )



In [2]:
# Drop NaNs
df_train.dropna(subset=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], inplace=True)

print("\n\nSummary statistics of training data")
print(df_train.describe())

# Age imputation
df_train.loc[df_train['Age'].isnull(), 'Age'] = np.nanmedian(df_train['Age'])
df_test.loc[df_test['Age'].isnull(), 'Age'] = np.nanmedian(df_test['Age'])

# Training/testing array creation
y_train, X_train = dmatrices('Survived ~ Age + Sex + Pclass + SibSp + Parch + Embarked', df_train)
X_test = dmatrix('Age + Sex + Pclass + SibSp + Parch + Embarked', df_test)

# Creating processing pipelines with preprocessing. Hyperparameters selected using cross validation
steps1 = [('poly_features', PolynomialFeatures(3, interaction_only=True)),
          ('logistic', LogisticRegression(C=5555., max_iter=16, penalty='l2'))]
steps2 = [('rforest', RandomForestClassifier(min_samples_split=15, n_estimators=73, criterion='entropy'))]
pipeline1 = Pipeline(steps=steps1)
pipeline2 = Pipeline(steps=steps2)

# Logistic model with cubic features
pipeline1.fit(X_train, y_train.ravel())
print('Accuracy (Logistic Regression-Poly Features (cubic)): {:.4f}'.format(pipeline1.score(X_train, y_train.ravel())))

# Random forest with calibration
pipeline2.fit(X_train[:600], y_train[:600].ravel())
calibratedpipe2 = CalibratedClassifierCV(pipeline2, cv=3, method='sigmoid')
calibratedpipe2.fit(X_train[600:], y_train[600:].ravel())
print('Accuracy (Random Forest - Calibration): {:.4f}'.format(calibratedpipe2.score(X_train, y_train.ravel())))

# Create the output dataframe
output = pd.DataFrame(columns=['PassengerId', 'Survived'])
output['PassengerId'] = df_test['PassengerId']

# Predict the survivors and output csv
output['Survived'] = pipeline1.predict(X_test).astype(int)
output.to_csv('output.csv', index=False)



Summary statistics of training data
       PassengerId    Survived      Pclass         Age       SibSp  \
count   712.000000  712.000000  712.000000  712.000000  712.000000   
mean    448.589888    0.404494    2.240169   29.642093    0.514045   
std     258.683191    0.491139    0.836854   14.492933    0.930692   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     222.750000    0.000000    1.000000   20.000000    0.000000   
50%     445.000000    0.000000    2.000000   28.000000    0.000000   
75%     677.250000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    5.000000   

            Parch        Fare  
count  712.000000  712.000000  
mean     0.432584   34.567251  
std      0.854181   52.938648  
min      0.000000    0.000000  
25%      0.000000    8.050000  
50%      0.000000   15.645850  
75%      1.000000   33.000000  
max      6.000000  512.329200  
Accuracy (Logistic Regression-Poly Features

In [ ]:
# Drop NaNs
df_train.dropna(subset=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], inplace=True)

print("\n\nSummary statistics of training data")
print(df_train.describe())

# Age imputation
df_train.loc[df_train['Age'].isnull(), 'Age'] = np.nanmedian(df_train['Age'])
df_test.loc[df_test['Age'].isnull(), 'Age'] = np.nanmedian(df_test['Age'])

# Training/testing array creation
y_train, X_train = dmatrices('Survived ~ Age + Sex + Pclass + SibSp + Parch + Embarked', df_train)
X_test = dmatrix('Age + Sex + Pclass + SibSp + Parch + Embarked', df_test)

# Creating processing pipelines with preprocessing. Hyperparameters selected using cross validation
steps1 = [('poly_features', PolynomialFeatures(3, interaction_only=True)),
          ('logistic', LogisticRegression(C=5555., max_iter=16, penalty='l2'))]
steps2 = [('rforest', RandomForestClassifier(min_samples_split=15, n_estimators=73, criterion='entropy'))]
pipeline1 = Pipeline(steps=steps1)
pipeline2 = Pipeline(steps=steps2)

# Logistic model with cubic features
pipeline1.fit(X_train, y_train.ravel())
print('Accuracy (Logistic Regression-Poly Features (cubic)): {:.4f}'.format(pipeline1.score(X_train, y_train.ravel())))

# Random forest with calibration
pipeline2.fit(X_train[:600], y_train[:600].ravel())
calibratedpipe2 = CalibratedClassifierCV(pipeline2, cv=3, method='sigmoid')
calibratedpipe2.fit(X_train[600:], y_train[600:].ravel())
print('Accuracy (Random Forest - Calibration): {:.4f}'.format(calibratedpipe2.score(X_train, y_train.ravel())))

# Create the output dataframe
output = pd.DataFrame(columns=['PassengerId', 'Survived'])
output['PassengerId'] = df_test['PassengerId']

# Predict the survivors and output csv
output['Survived'] = pipeline1.predict(X_test).astype(int)
output.to_csv('output.csv', index=False)